In [2]:
import math
import numpy as np
import pandas as pd
import scipy.stats as scs
import matplotlib.pyplot as plt
import matplotlib as mpl

In [3]:
from cmath import exp


def simulate_gbm():
    S0=100
    T=10
    I=1
    r=0.02
    vol=0.20
    np.random.seed(20000)
    gbm_dates=pd.date_range(start='30-09-2004',end='30-09-2014',freq='B')
    M=len(gbm_dates)
    dt=1/252
    df=np.exp(-r*dt)
    rand=np.random.standard_normal((M,I))
    S=np.zeros_like(rand)
    S[0]=S0
    for t in range(1,M):
       S[t]=S[t-1]*np.exp((r-vol**2/2)*dt+vol*rand[t]*math.sqrt(dt))
    gbm = pd.DataFrame(S[:,0], index=gbm_dates, columns=['index'])
    gbm['returns'] = np.log(gbm['index'] / gbm['index'].shift(1))
   # Realized Volatility (eg. as defined for variance swaps)
    gbm['rea_var'] = 252 * np.cumsum(gbm['returns'] ** 2) / np.arange(len(gbm))
    gbm['rea_vol'] = np.sqrt(gbm['rea_var'])
    gbm = gbm.dropna()
    return gbm

In [4]:
simulate_gbm()

/tmp/ipykernel_3909/520863277.py:11: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  gbm_dates=pd.date_range(start='30-09-2004',end='30-09-2014',freq='B')


,index,returns,rea_var,rea_vol
2004-10-01,99.620338,-0.003804,0.003646,0.060384
2004-10-04,98.599472,-0.010300,0.015192,0.123254
2004-10-05,97.889090,-0.007231,0.014520,0.120497
2004-10-06,96.641549,-0.012826,0.021254,0.145788
2004-10-07,94.356633,-0.023927,0.045858,0.214145
...,...,...,...,...
2014-09-24,70.798218,-0.005110,0.040225,0.200562
2014-09-25,70.861515,0.000894,0.040210,0.200523
2014-09-26,71.922306,0.014859,0.040216,0.200538
2014-09-29,71.111182,-0.011342,0.040213,0.200531


In [5]:
def print_statistics(data):
    print ("RETURN SAMPLE STATISTICS")
    print ("---------------------------------------------")
    print ("Mean of Daily Log Returns %9.6f" % (np.mean(data['returns'])))
    print ("Std of Daily Log Returns %9.6f" % np.std(data['returns']))
    print ("Mean of Annua. Log Returns %9.6f" % (np.mean(data['returns']) * 252))
    print ("Std of Annua. Log Returns %9.6f" % (np.std(data['returns']) * math.sqrt(252)))
    print ("---------------------------------------------")
    print ("Skew of Sample Log Returns %9.6f" % scs.skew(data['returns']))
    print ("Skew Normal Test p-value %9.6f" % scs.skewtest(data['returns'])[1])
    print ("---------------------------------------------")
    print ("Kurt of Sample Log Returns %9.6f" % scs.kurtosis(data['returns']))
    print ("Kurt Normal Test p-value %9.6f" % \
    scs.kurtosistest(data['returns'])[1])
    print ("---------------------------------------------")
    print ("Normal Test p-value %9.6f" % \
    scs.normaltest(data['returns'])[1])
    print ("---------------------------------------------")
    print ("Realized Volatility %9.6f" % data['rea_vol'].iloc[-1])
    print ("Realized Variance %9.6f" % data['rea_var'].iloc[-1])  

In [6]:
print_statistics(simulate_gbm())

RETURN SAMPLE STATISTICS
---------------------------------------------
Mean of Daily Log Returns -0.000134
Std of Daily Log Returns  0.012630
Mean of Annua. Log Returns -0.033660
Std of Annua. Log Returns  0.200494
---------------------------------------------
Skew of Sample Log Returns -0.042229
Skew Normal Test p-value  0.377481
---------------------------------------------
Kurt of Sample Log Returns -0.074954
Kurt Normal Test p-value  0.457202
---------------------------------------------
Normal Test p-value  0.513856
---------------------------------------------
Realized Volatility  0.200505
Realized Variance  0.040202


/tmp/ipykernel_3909/520863277.py:11: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  gbm_dates=pd.date_range(start='30-09-2004',end='30-09-2014',freq='B')
